In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import random
import lyricsgenius
from datetime import datetime
import pylast

# Set up authentication credentials
client_id = '24458f331f214f5fb51d9c1038d051a2'
client_secret = '763bb85ec2b34d7da053f7a3d40a8a35'
lastfm_api_key = 'c6eadef00fb0d64d94e4971f325ad128'
lastfm_api_secret = '7320cd2fb3a76ba15647ffa992d41875'


In [3]:
network = pylast.LastFMNetwork(api_key=lastfm_api_key, api_secret=lastfm_api_secret)
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
genius = lyricsgenius.Genius('JIjj-ZpDdQVuwBIZJcxygMNFGHJDkondzLx6kbyrZcQQbQV23guBwXMgomgU2BXt')

In [13]:
# results = sp.search(q='year:2023', type='track', limit=50, offset=0)
# tracks = results['tracks']['items']
# track_ids = [t['id'] for t in tracks]

# Retrieve track information for each track ID
energys = []
trackNames = []
artists = []
genres = []
lengths = []
tempos = []
lyrics = []
releasedates = []
releasemonths = []
popularities = []

desired_limit = 90
total_responses = 0
offset = 0
track_ids = []

# Make multiple requests to retrieve the desired number of responses
while total_responses < desired_limit:
    # Determine the remaining number of responses to retrieve
    remaining_limit = desired_limit - total_responses
    # Adjust the request limit based on the remaining responses
    request_limit = min(50, remaining_limit)

    # Retrieve tracks with the specified limit and offset
    results = sp.search(q='year:2022', type='track', limit=request_limit, offset=offset)
    tracks = results['tracks']['items']
    track = [t['id'] for t in tracks]
    # Append the retrieved tracks to the list
    track_ids.extend(track)

    # Update the total number of responses and offset for the next request
    total_responses += len(tracks)
    offset += request_limit

In [14]:
for track_id in track_ids:
    # Wait for a random amount of time to avoid hitting API rate limits
    time.sleep(random.uniform(0.5, 1.5))

    # Retrieve track information
    track = sp.track(track_id)
    #print(track)
    if track is None:
        continue
    features = sp.audio_features(track_id)
    print(features)
    if (features == [None]):
        continue
    else:
        tempo = features[0]['tempo']
    # if album is None:
    #     continue
    # if album['genres'] == []:
    #     genre = None
    # else:
    #     genre = album['genres'][0]



    length = track['duration_ms']
    # tempo = features[0]['tempo'] if 'tempo' in features[0] else None
    popularity = track['popularity']
    timeSinceRelease = datetime.today() - datetime.strptime(track['album']['release_date'], '%Y-%m-%d')
    month = datetime.strptime(track['album']['release_date'], '%Y-%m-%d').month
    #energy = features[0]['energy']
    
    

    # Retrieve lyrics using the Genius API
    # song = genius.search_song(track['name'], track['artists'][0]['name'])
    # if song is not None:
    #     lyrics.append(song.lyrics)
    # else:
    #     lyrics.append(None)

    # Append to lists
    # genres.append(genre)
    # print(tempo)
    releasemonths.append(month)
    releasedates.append(timeSinceRelease.days)
    lengths.append(length)
    tempos.append(tempo)
    popularities.append(popularity)
    artists.append(track['artists'][0]['name'])
    trackNames.append(track['name'])

[{'danceability': 0.715, 'energy': 0.62, 'key': 1, 'loudness': -6.005, 'mode': 0, 'speechiness': 0.0484, 'acousticness': 0.417, 'instrumentalness': 0, 'liveness': 0.0822, 'valence': 0.172, 'tempo': 97.95, 'type': 'audio_features', 'id': '2dHHgzDwk4BJdRwy9uXhTO', 'uri': 'spotify:track:2dHHgzDwk4BJdRwy9uXhTO', 'track_href': 'https://api.spotify.com/v1/tracks/2dHHgzDwk4BJdRwy9uXhTO', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2dHHgzDwk4BJdRwy9uXhTO', 'duration_ms': 221520, 'time_signature': 4}]
[{'danceability': 0.644, 'energy': 0.735, 'key': 8, 'loudness': -5.747, 'mode': 1, 'speechiness': 0.0391, 'acousticness': 0.0521, 'instrumentalness': 0.144, 'liveness': 0.161, 'valence': 0.418, 'tempo': 88.98, 'type': 'audio_features', 'id': '1Qrg8KqiBpW07V7PNxwwwL', 'uri': 'spotify:track:1Qrg8KqiBpW07V7PNxwwwL', 'track_href': 'https://api.spotify.com/v1/tracks/1Qrg8KqiBpW07V7PNxwwwL', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1Qrg8KqiBpW07V7PNxwwwL', 'duration_ms':

In [15]:
print(len(popularities))

997


In [10]:
data = pd.DataFrame({'length': lengths, 'tempo': tempos, 'timeSinceRelease':releasedates, 'releaseMonth':releasemonths, 'artists':artists, 'names':trackNames, 'popularity': popularities})

In [15]:
# genre_dummies = pd.get_dummies(data['genre'].apply(pd.Series).stack()).sum()
# data = pd.concat([data, genre_dummies], axis=1).drop('genre', axis=1)

In [12]:
data.to_csv('spotify_data_2022.csv', index=False)